In [19]:
#导入需要的包
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import paddle
print("本教程基于Paddle的版本号为："+paddle.__version__)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


本教程基于Paddle的版本号为：2.1.0


In [20]:
#导入数据集Compose的作用是将用于数据集预处理的接口以列表的方式进行组合。
#导入数据集Normalize的作用是图像归一化处理，支持两种方式： 1. 用统一的均值和标准差值对图像的每个通道进行归一化处理； 2. 对每个通道指定不同的均值和标准差值进行归一化处理。
from paddle.vision.transforms import Compose, Normalize, Resize, RandomRotation, RandomCrop
img_size = 32
#对训练集做数据增强
transform1 = Compose([Resize((img_size+2, img_size+2)), RandomCrop(img_size), Normalize(mean=[127.5],std=[127.5],data_format='CHW')])
transform2 = Compose([Resize((img_size, img_size)), Normalize(mean=[127.5],std=[127.5],data_format='CHW')])
# 使用transform对数据集做归一化
print('下载并加载训练数据')
train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=transform1)
test_dataset = paddle.vision.datasets.MNIST(mode='test', transform=transform2)
print('加载完成')


下载并加载训练数据


Cache file /home/aistudio/.cache/paddle/dataset/mnist/train-images-idx3-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/train-images-idx3-ubyte.gz 
Begin to download

Download finished
Cache file /home/aistudio/.cache/paddle/dataset/mnist/train-labels-idx1-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/train-labels-idx1-ubyte.gz 
Begin to download
........
Download finished
Cache file /home/aistudio/.cache/paddle/dataset/mnist/t10k-images-idx3-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/t10k-images-idx3-ubyte.gz 
Begin to download

Download finished
Cache file /home/aistudio/.cache/paddle/dataset/mnist/t10k-labels-idx1-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/t10k-labels-idx1-ubyte.gz 
Begin to download
..
Download finished


加载完成


In [21]:
import paddle
import paddle.nn.functional as F
# 定义多层卷积神经网络
#动态图定义多层卷积神经网络
class ResNet(paddle.nn.Layer):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = paddle.nn.Conv2D(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.conv2 = paddle.nn.Conv2D(6, 16, 3, padding=1)
        self.conv3 = paddle.nn.Conv2D(16, 32, 3, padding=1)
        self.conv4 = paddle.nn.Conv2D(6, 32, 1)

        self.conv5 = paddle.nn.Conv2D(32, 64, 3, padding=1)
        self.conv6 = paddle.nn.Conv2D(64, 128, 3, padding=1)
        self.conv7 = paddle.nn.Conv2D(32, 128, 1)

        self.maxpool1 = paddle.nn.MaxPool2D(kernel_size=2, stride=2)
        
        self.maxpool2 = paddle.nn.MaxPool2D(2, 2)
        self.maxpool3 = paddle.nn.MaxPool2D(2, 2)
        self.maxpool4 = paddle.nn.MaxPool2D(2, 2)
        self.maxpool5 = paddle.nn.MaxPool2D(2, 2)
        self.maxpool6 = paddle.nn.MaxPool2D(2, 2)
        self.flatten=paddle.nn.Flatten()
        self.linear1=paddle.nn.Linear(128, 128)
        self.linear2=paddle.nn.Linear(128, 10)
        self.dropout=paddle.nn.Dropout(0.2)
        self.avgpool=paddle.nn.AdaptiveAvgPool2D(output_size=1)
        
    def forward(self, x):
        y = self.conv1(x)#(bs 6, 32, 32)
        y = F.relu(y)
        y = self.maxpool1(y)#(bs, 6, 16, 16)
        z = y
        y = self.conv2(y)#(bs, 16, 16, 16)
        y = F.relu(y)
        y = self.maxpool2(y)#(bs, 16, 8, 8)
        y = self.conv3(y)#(bs, 32, 8, 8)
        z = self.maxpool4(self.conv4(z))
        y = y+z
        y = F.relu(y)
        z = y
        y = self.conv5(y)#(bs, 64, 8, 8)
        y = F.relu(y)
        y = self.maxpool5(y)#(bs, 64, 4, 4)
        y = self.conv6(y)#(bs, 128, 4, 4)
        z = self.maxpool6(self.conv7(z))
        y = y + z
        y = F.relu(y)
        y = self.avgpool(y)
        y = self.flatten(y)
        y = self.linear1(y)
        y = self.dropout(y)
        y = self.linear2(y)
        return y

In [22]:
#定义卷积网络的代码
net_cls = ResNet()
paddle.summary(net_cls, (-1, 1, img_size, img_size))

-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
     Conv2D-1         [[1, 1, 32, 32]]      [1, 6, 32, 32]          156      
    MaxPool2D-1       [[1, 6, 32, 32]]      [1, 6, 16, 16]           0       
     Conv2D-2         [[1, 6, 16, 16]]     [1, 16, 16, 16]          880      
    MaxPool2D-2      [[1, 16, 16, 16]]      [1, 16, 8, 8]            0       
     Conv2D-3         [[1, 16, 8, 8]]       [1, 32, 8, 8]          4,640     
     Conv2D-4         [[1, 6, 16, 16]]     [1, 32, 16, 16]          224      
    MaxPool2D-4      [[1, 32, 16, 16]]      [1, 32, 8, 8]            0       
     Conv2D-5         [[1, 32, 8, 8]]       [1, 64, 8, 8]         18,496     
    MaxPool2D-5       [[1, 64, 8, 8]]       [1, 64, 4, 4]            0       
     Conv2D-6         [[1, 64, 4, 4]]       [1, 128, 4, 4]        73,856     
     Conv2D-7         [[1, 32, 8, 8]]       [1, 128, 8, 8]    

{'total_params': 120278, 'trainable_params': 120278}

In [23]:
from paddle.metric import Accuracy
save_dir = "output/model5_7"
patience = 5
epoch = 20
lr = 0.01
weight_decay = 5e-4
batch_size = 64
momentum = 0.9
# 用Model封装模型
model = paddle.Model(net_cls)

# 定义损失函数
#optim = paddle.optimizer.AdamW(learning_rate=lr, parameters=model.parameters(), weight_decay=weight_decay)
#lr = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=lr, T_max=10000, eta_min=1e-5)
optim = paddle.optimizer.Momentum(learning_rate=lr, parameters=model.parameters(), momentum=momentum)

visual_dl = paddle.callbacks.VisualDL(log_dir=save_dir)
early_stop = paddle.callbacks.EarlyStopping(monitor='acc', mode='max', patience=patience, 
                                            verbose=1, min_delta=0, baseline=None,
                                            save_best_model=True)
# 配置模型
model.prepare(optim,paddle.nn.CrossEntropyLoss(),Accuracy())

# 训练保存并验证模型
model.fit(train_dataset,test_dataset,epochs=epoch,batch_size=batch_size,
        save_dir=save_dir,verbose=1, callbacks=[visual_dl, early_stop])

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/20


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 938/938 [==============================] - loss: 0.0568 - acc: 0.8598 - 80ms/step         
save checkpoint at /home/aistudio/output/model5_7/0
Eval begin...
step 157/157 [==============================] - loss: 0.0047 - acc: 0.9769 - 24ms/step        
Eval samples: 10000
Epoch 2/20
step 938/938 [==============================] - loss: 0.0525 - acc: 0.9704 - 80ms/step         
save checkpoint at /home/aistudio/output/model5_7/1
Eval begin...
step 157/157 [==============================] - loss: 0.0022 - acc: 0.9820 - 24ms/step          
Eval samples: 10000
Epoch 3/20
step 938/938 [==============================] - loss: 0.0115 - acc: 0.9787 - 80ms/step        
save checkpoint at /home/aistudio/output/model5_7/2
Eval begin...
step 157/157 [==============================] - loss: 0.0022 - acc: 0.9876 - 24ms/step          
Eval samples: 10000
Epoch 4/20
step 938/938 [==============================] - loss: 0.0188 - acc: 0.9836 - 80ms/step         
save checkpoint at /home/aistudio/out

In [26]:
best_model_path = "output/model5_7/best_model.pdparams"
net_cls = ResNet()
model = paddle.Model(net_cls)
model.load(best_model_path)
model.prepare(optim,paddle.nn.CrossEntropyLoss(),Accuracy())


In [27]:
#用最好的模型在测试集10000张图片上验证
results = model.evaluate(test_dataset, batch_size=batch_size, verbose=1)
print(results)

Eval begin...
step 157/157 [==============================] - loss: 5.2440e-05 - acc: 0.9935 - 25ms/step      
Eval samples: 10000
{'loss': [5.243971e-05], 'acc': 0.9935}


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 